# Pulling Tweets from WHO

In [1]:
# Import libbraries
import os, sys, subprocess
import json
import pandas as pd
import numpy as np
from langdetect import detect
import pickle

pd.set_option('display.max_colwidth', -1)

pd.set_option('display.max_rows', 1000)
#pd.set_option('display.max_columns', 500)

In [2]:
# load project config

terminal_call = ! git rev-parse --show-toplevel
repo_path=terminal_call[0]
project_config_path = os.path.join(repo_path,'project_config.json')

with open(project_config_path,'r') as fp: 
    project_config = json.load(fp)


project_config

{'project_module_relative_path': 'src'}

In [3]:
# import custom scraper module

module_path = os.path.join(repo_path,project_config['project_module_relative_path'])
sys.path.append(module_path)

import scraper
from scraper import tweet_scraper

from importlib import reload

In [37]:
reload(tweet_scraper)

<module 'scraper.tweet_scraper' from '/Users/vivianpeng/git/tweet_analysis/src/scraper/tweet_scraper.py'>

## Pull Tweets for a year

In [54]:
# get tweets from for a year from this date
df = tweet_scraper.query_tweets('global common goods', '2019-06-01', '2020-06-01')

In [55]:
# rename columns
df.rename(columns={0:"date",
          1:"text",
          2: "user"}, inplace = True)

In [56]:
# set column types
df['text'] = df['text'].astype(str)
df['date'] = pd.to_datetime(df['date'])
df['user'] = df['user'].astype(str)

In [57]:
# quick sort on most common users
df.groupby("user").count().sort_values("text", ascending=False).head(20)

,date,text
user,,
GYamey,9,9
asoucat,7,7
DukeCPIGH,6,6
benavides_marta,5,5
DebSharp8,5,5
UHC2030,4,4
JTasioulas,4,4
Rob_Kimbell,3,3
ghia1212,2,2


In [58]:
df.shape

(195, 3)

## Pickle for later use

In [29]:
# pickle.dump(df, open("../data/gpg.pkl", "wb"))

## Clean Tweets

In [23]:
# Reload package when I make updates to function
# reload(tweet_scraper)

In [59]:
# Clean tweet column
df = tweet_scraper.clean_tweets(df, 'text')

# Write to CSV

In [60]:
# write to csv
df.to_csv("../data/gcgs.csv", index = False)